# Ex21



Uma empresa deseja determinar o plano de investimentos para o próximo ano, e dispõe dos seguintes projetos:

| **Possibilidade de Investimento** | **Valor Presente Líquido (US$)** | **Capital Requerido (US$)** |
|-----------------------------------|----------------------------------|----------------------------|
| I Construir um novo depósito      | 7.000.000,00                     | 5.000.000,00               |
| II Recuperar o depósito antigo   | 4.500.000,00                     | 3.000.000,00               |
| III Automatizar o depósito novo  | 5.500.000,00                     | 4.200.000,00               |
| IV Comprar a fornecedora do produto A | 12.000.000,00               | 9.300.000,00               |
| V Construir uma fábrica para produzir A | 9.500.000,00             | 7.100.000,00               |
| VI Reformar o escritório da empresa | 1.500.000,00                 | 900.000,00                 |

### **Restrições**
- As alternativas **I e II** são mutuamente excludentes.
- As alternativas **IV e V** são mutuamente excludentes.
- O projeto **III depende da realização do projeto I**.
- O orçamento disponível para investimento é **US$ 20.000.000,00**.

Formule um modelo de **programação inteira binária** para determinar o portfólio ótimo de investimento.









### **Definições e Variáveis**
- $ x_i $ : variável binária indicando se o projeto $ i $ é escolhido (1) ou não (0), para $ i = \{I, II, III, IV, V, VI\} $.
- $ C_i $ : capital requerido para o projeto $ i $.
- $ VPL_i $ : valor presente líquido associado ao projeto $ i $.
- O orçamento disponível é $ B = 20.000.000 $.

### **Função Objetivo**
Maximizar o valor presente líquido total:
$$ \max \sum_{i \in \{I, II, III, IV, V, VI\}} VPL_i x_i $$

### **Restrições**
1. **Restrição de orçamento:**  
   $$ \sum_{i \in \{I, II, III, IV, V, VI\}} C_i x_i \leq B $$

2. **Mutual exclusividade entre os projetos I e II:**  
   $$ x_I + x_{II} \leq 1 $$

3. **Mutual exclusividade entre os projetos IV e V:**  
   $$ x_{IV} + x_V \leq 1 $$

4. **Dependência do projeto III em relação ao projeto I:**  
   $$ x_{III} \leq x_I $$

5. **Variáveis binárias:**  
   $$ x_i \in \{0,1\}, \quad \forall i $$



In [16]:
import pyomo.environ as pyo

In [17]:
# Criação do modelo
model = pyo.ConcreteModel()

In [18]:

# Conjuntos
model.I = pyo.Set(initialize=["I", "II", "III", "IV", "V","VI"])



In [19]:
# Valor presente líquido (VPL) de cada projeto (R$ milhões)
vpl = {"I": 8, "II": 7, "III": 5, "IV": 6, "V": 4, "VI": 9}
model.vpl = pyo.Param(model.I, initialize=vpl)

# Capital requerido para cada projeto (R$ milhões)
custo = {"I": 6, "II": 5, "III": 3, "IV": 4, "V": 3, "VI": 7}
model.custo = pyo.Param(model.I, initialize=custo)

# Orçamento disponível (R$ milhões)
budget = 20

In [20]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x =  pyo.Var(model.I, domain=pyo.Binary)

In [21]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.vpl[i] * model.x[i] for i in model.I)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [22]:
# ------------------------------
# Restrições
# ------------------------------
# 1. Restrição de orçamento
def restricao_orcamento(model):
    return sum(model.custo[i] * model.x[i] for i in model.I) == budget

model.restricao_orcamento = pyo.Constraint(rule=restricao_orcamento)

# 2. Mutual exclusividade entre os projetos I e II
def exclusividade_I_II(model):
    return model.x["I"] + model.x["II"] <= 1

model.exclusividade_I_II = pyo.Constraint(rule=exclusividade_I_II)

# 3. Mutual exclusividade entre os projetos IV e V
def exclusividade_IV_V(model):
    return model.x["IV"] + model.x["V"] <= 1

model.exclusividade_IV_V = pyo.Constraint(rule=exclusividade_IV_V)

# 4. Dependência do projeto III em relação ao projeto I
def dependencia_III_I(model):
    return model.x["III"] <= model.x["I"]

model.dependencia_III_I = pyo.Constraint(rule=dependencia_III_I)



In [23]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex21.lp", io_options={"symbolic_solver_labels": True})

('ex21.lp', 2014574235024)

In [24]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 4 rows; 6 cols; 12 nonzeros; 6 integer variables (6 binary)
Coefficient ranges:
  Matrix [1e+00, 7e+00]
  Cost   [4e+00, 9e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+01]
Presolving model
4 rows, 6 cols, 12 nonzeros  0s
4 rows, 6 cols, 12 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   4 rows
   6 cols (6 binary, 0 integer, 0 implied int., 0 continuous)
   12 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.

In [25]:
print("\nProjetos escolhidos para investimento:")
for i in model.I:
    if model.x[i].value > 0.5:
        print(f"Projeto {i}")

print("\nVPL total maximizado:")
print(f"R$ {model.objetivo()} milhões")


Projetos escolhidos para investimento:
Projeto I
Projeto III
Projeto IV
Projeto VI

VPL total maximizado:
R$ 28.0 milhões
